In [ ]:
##### This code is used for obtaining BBB disruption model data with stochastic model #####
if 'RunningFromFigureScript' in locals(): # Checking where from the script was runned. If from
    # figure, the directory path is then different
    %run ./DataGenerationScripts/load_default_parameters.ipynb # loading model parameters
    %run ./DataGenerationScripts/dIBDRdt_functions.ipynb # loading time derivative functions
else:
    %reset 
    %run ../DataGenerationScripts/load_default_parameters.ipynb # loading model parameters
    %run ../DataGenerationScripts/dIBDRdt_functions.ipynb # loading time derivative functions 
    
import numpy as np
from tqdm import tqdm

In [ ]:
 # In this cell we define simulation-specific parameters of simulation 
        # duration; external input parameters; number of simulation; output location for data  
params=load_default_parameters('stochastic')
params['t_end']= 90          
params['IBDR_E_duration']= [0,7/2,0,0]
params['IBDR_E_amplitude']= [0,0.25,0,0]
params['Complex_input']= 'no' 
params['amount_simulations']=  60
params['number_simulation']= 0
if 'RunningFromFigureScript' in locals(): # Checking where from the script was runned and creating respective figure path
    params['path']= './Data/Suppl_Data_FS11_Stoch_BBBm_short/Sim_'
else:
    params['path']= '../Data/Suppl_Data_FS11_Stoch_BBBm_short/Sim_'

In [ ]:
locals().update(params)
np.random.seed(seed_value)

t_vec=np.linspace(0,t_end,int(t_end/dt+1))
I_vec=np.zeros(np.shape(t_vec))
B_vec=np.zeros(np.shape(t_vec))
D_vec=np.zeros(np.shape(t_vec))
R_vec=np.zeros(np.shape(t_vec))
S_vec=np.zeros(np.shape(t_vec)) # seizure vector: 1- seizure onset, 2-seizure is still active (for cases T_seiz>dt)
lambda_vec=np.zeros(np.shape(t_vec)) # seiz rate vector

t_lastseiz = []
I_vec[0]=IC[0] #  setting initial conditions
B_vec[0]=IC[1]
D_vec[0]=IC[2]
R_vec[0]=IC[3]

for sim_number in tqdm(range(1, amount_simulations + 1)): # performing N simulations
    params['number_simulation'] = sim_number
    
    
    for ii in range(1, len(t_vec)): # solving with forward Euler method  
        [dIdt, dBdt, dDdt, dRdt, S_vec[ii], t_lastseiz] = \
                dIBDRdt_Stoch(t_vec[ii], I_vec[ii-1], B_vec[ii-1], D_vec[ii-1], R_vec[ii-1], S_vec[ii-1], lambda_vec[ii-1], t_lastseiz , params)
                
        I_vec[ii] = I_vec[ii-1] + dIdt * dt
        B_vec[ii] = B_vec[ii-1] + dBdt * dt
        D_vec[ii] = D_vec[ii-1] + dDdt * dt
        R_vec[ii] = R_vec[ii-1] + dRdt * dt

        lambda_vec[ii]=SeizureRate(I_vec[ii],R_vec[ii], params)

    results_dict = {'t_vec': t_vec,
                    'I_vec': I_vec,
                    'B_vec': B_vec,
                    'D_vec': D_vec,
                    'R_vec': R_vec,
                    'S_vec': S_vec,
                    'lambda_vec': lambda_vec}

    filename = path + str(sim_number)
    np.save(filename, [params, results_dict])
    del ii, filename